In [1]:
import pickle
import bson
import pandas as pd
import pandas as pd
import numpy as np
import os
import pickle
from bson.objectid import ObjectId
from collections import defaultdict
import timeit
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import datetime


pd.set_option('display.max_columns', None)

In [5]:
print(os.getcwd())

c:\Users\porwa\OneDrive - Georgia Institute of Technology\Desktop\GT Acad\09 Fall 2023\01 Practicum\07 NCAAW


In [7]:
df_shots = pd.read_pickle("NCAAW1+ 22-23 shotsdf.pickle")
df_games = pd.read_pickle("NCAAW1+ 22-23 gamesdf.pickle")
df_players = pd.read_pickle("NCAAW1+ 22-23 playerdictionary.pickle")
df_players = pd.DataFrame(df_players)
df_team = pd.read_pickle("NCAAW1+ 22-23 teamsdf.pickle")
df_games_prev_season = pd.read_pickle("NCAAW1+ 21-22 gamesdf.pickle")

In [8]:
features_df = df_shots[['game_id', 'team_id', 'season_id', 'player_id', 'shot_id','segment_id','Distance', 'Angle', 'Zone', 'x_coordinate', 'y_coordinate', 'Period','Time_Seconds','Time','score','score_opp','Stat']].copy()

# For shot/play type, we'll take multiple columns
shot_play_type_columns = ['Jumpshot', 'Assisted', 'Red Zone', 'Floater', 'Second Chance', 'Layup', 'Paint', 'Drive', 'Blocked', 'ATO', 'Off Turnover', 'Off Steal', 'Hook Shot', 'Fastbreak', 'Pullup', 'And1', 'Turnaround', 'Dunk', 'Step Back']
features_df[shot_play_type_columns] = df_shots[shot_play_type_columns]

In [9]:
merged_df = pd.merge(features_df, df_games, on='game_id', how='left')

# Create 'Location' feature based on 'team_id' and 'hometeam'
def determine_location(row):
    if pd.isna(row['hometeam']):
        return 'Neutral'
    elif row['team_id'] == row['hometeam']:
        return 'Home'
    else:
        return 'Away'

merged_df['Location'] = merged_df.apply(determine_location, axis=1)

In [10]:
df_games['date'] = pd.to_datetime(df_games['date'])
merged_df['date'] = pd.to_datetime(merged_df['date'])

C:\Users\porwa\AppData\Local\Temp\ipykernel_34756\2950306567.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_games['date'] = pd.to_datetime(df_games['date'])
C:\Users\porwa\AppData\Local\Temp\ipykernel_34756\2950306567.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['date'] = pd.to_datetime(merged_df['date'])


In [11]:
periods = [10, 5, 3, 1]

def compute_win_loss_ratios_for_period(df_games, team_id, period):
    """Compute win-loss ratios for a team for the given period."""
    # Filter the team's games
    team_games = df_games[(df_games['team1'] == team_id) | (df_games['team2'] == team_id)]
    
    # Sort by date
    team_games = team_games.sort_values(by='date')
    
    # Compute win flags (1 if won, 0 otherwise)
    team_games['won'] = (team_games['winning_team'] == team_id).astype(int)
    
    # Calculate rolling win sum and count, excluding the current game
    team_games[f'wins_last_{period}'] = team_games['won'].shift(1).rolling(window=period, min_periods=1, closed='left').sum()
    team_games[f'games_last_{period}'] = team_games['won'].shift(1).rolling(window=period, min_periods=1, closed='left').count()
    
    # Calculate ratio
    team_games[f'w_l_ratio_last_{period}'] = team_games[f'wins_last_{period}'] / team_games[f'games_last_{period}']
    
    return team_games[['date', f'w_l_ratio_last_{period}']].set_index('date')

# Calculate win-loss ratios for all teams and periods in advance
team_ids = df_games['team1'].unique()
ratios_cache = {}
for team_id in team_ids:
    for period in periods:
        ratios_cache[(team_id, period)] = compute_win_loss_ratios_for_period(df_games, team_id, period)

def get_opponent(row):
    return row['team2'] if row['team_id'] == row['team1'] else row['team1']

from tqdm import tqdm

def get_ratio_from_cache_vectorized(dates, team_ids, period):
    """Vectorized function to fetch pre-calculated ratio from the cache."""
    results = []
    for date, team_id in zip(dates, team_ids):
        ratios = ratios_cache.get((team_id, period))
        if ratios is not None:
            results.append(ratios.loc[date, f'w_l_ratio_last_{period}'] if date in ratios.index else float('nan'))
        else:
            results.append(float('nan'))
    return results

for period in tqdm(periods, desc='Updating DataFrame'):
    # Team stats
    merged_df[f'w_l_ratio_last_{period}'] = get_ratio_from_cache_vectorized(merged_df['date'], merged_df['team_id'], period)
    # Opposition stats
    merged_df[f'opp_w_l_ratio_last_{period}'] = get_ratio_from_cache_vectorized(merged_df['date'], merged_df.apply(get_opponent, axis=1), period)

Updating DataFrame: 100%|██████████| 4/4 [02:25<00:00, 36.27s/it]


In [12]:
# Remove rows with no player_id
merged_df = merged_df[merged_df.player_id != False]

# Remove all the corner 3s that are registered as 2s
merged_df = merged_df[~(((merged_df.Zone == ' 4-1') & (merged_df.x_coordinate <= 3)) |
                         ((merged_df.Zone == ' 4-3') & (merged_df.x_coordinate >= 47)))]

# Remove all the above the break 3s that are registered as 2s
merged_df = merged_df[~(((merged_df.Zone == ' 4-1') | (merged_df.Zone == ' 4-3') | (merged_df.Zone == ' 4-2')) &
                         (merged_df.Distance >= 22.1458))]

# Remove all the 2s that are registered as above the break 3s
merged_df = merged_df[~(((merged_df.Zone == ' 6-1') | (merged_df.Zone == ' 6-3') | (merged_df.Zone == ' 6-2')) &
                         (merged_df.Distance < 22.1458))]

# Remove all the 2s that are registered as corner 3s
merged_df = merged_df[~(((merged_df.Zone == ' 5-1') & (merged_df.x_coordinate > 3)) |
                         ((merged_df.Zone == ' 5-2') & (merged_df.x_coordinate < 47) & (merged_df.x_coordinate > 3)))]

# Reclassify the misclassified corner 3s (left instead of right corner)
merged_df.loc[((merged_df.Zone == ' 5-2') & (merged_df.x_coordinate <= 3)), 'Zone'] = ' 5-1'

Merging Saksham's features

In [13]:
df_features_additional = pd.read_pickle("features_SA_NCAAW1+.pickle")
print(df_features_additional.shape)
print(merged_df.shape)
merged_df = pd.merge(merged_df, df_features_additional, on=['shot_id'], how='left')
print(merged_df.shape)

(664654, 14)
(664654, 54)
(664654, 67)


Adding player attributes

In [14]:
merged_df = pd.merge(merged_df, df_players, left_on='player_id', right_on='player', how='left')
print(merged_df.shape)
merged_df.drop(columns=['player', 'Genius_position','seasonid'], inplace=True)
print(merged_df.shape)

(664654, 73)
(664654, 70)


Adding previous season win-loss ratio

In [15]:
def compute_prev_season_win_loss_ratio(df_games_prev_season):
    team_wins = df_games_prev_season.groupby('winning_team').size()
    team_games = df_games_prev_season.groupby(['team1']).size() + df_games_prev_season.groupby(['team2']).size()

    # Win-loss ratio
    win_loss_ratio = team_wins / team_games
    return win_loss_ratio

win_loss_ratio_prev_season = compute_prev_season_win_loss_ratio(df_games_prev_season)

def get_team_wl_ratio(team_id, win_loss_ratio):
    return win_loss_ratio.get(team_id, float('nan'))

merged_df['team_wl_ratio_prev_season'] = merged_df['team_id'].apply(lambda x: get_team_wl_ratio(x, win_loss_ratio_prev_season))
merged_df['opp_team_wl_ratio_prev_season'] = merged_df.apply(lambda row: get_team_wl_ratio(get_opponent(row), win_loss_ratio_prev_season), axis=1)

Splitting into 2 pointer & 3 pointer datasets

In [16]:
merged_df_two_pointers = merged_df[merged_df['Stat'].isin(['TwoMake', 'TwoMiss'])].copy()
merged_df_three_pointers = merged_df[merged_df['Stat'].isin(['ThreeMake', 'ThreeMiss'])].copy()

merged_df_two_pointers['Made'] = merged_df_two_pointers['Stat'] == 'TwoMake'
merged_df_three_pointers['Made'] = merged_df_three_pointers['Stat'] == 'ThreeMake'

Adding Nikos's features

In [17]:
feature_df_3P = pd.read_pickle('Condensed_3P_features_W_df.pickle')
feature_df_2P = pd.read_pickle('Condensed_2P_features_W_df.pickle')

print(feature_df_3P.shape)
print(feature_df_2P.shape)

(216949, 75)
(447705, 75)


In [18]:
merged_df_with_2P = pd.merge(merged_df_two_pointers, feature_df_2P, on=['shot_id'], how='left')
merged_df_with_3P = pd.merge(merged_df_three_pointers, feature_df_3P, on=['shot_id'], how='left')

print(merged_df_with_2P.shape)
print(merged_df_with_3P.shape)

(447705, 147)
(216949, 147)


In [19]:
merged_df_with_2P.to_csv('Women_merged_df_with_2P.csv')
merged_df_with_3P.to_csv('Women_merged_df_with_3P.csv')